# Functional API Preview

This notebook is part of a set of notebooks that provides a preview of the proposed functional API for dowhy. For details on the new API for DoWhy, check out https://github.com/py-why/dowhy/wiki/API-proposal-for-v1 It is a work-in-progress and is updated as we add new functionality. We welcome your feedback through Discord or on the Discussions page.
This functional API is designed with backwards compatibility. So both the old and new API will continue to co-exist and work for the immediate new releases. Gradually the old API using CausalModel will be deprecated in favor of the new API. 

The current Functional API covers:
* Identify Effect:
  * `identify_effect(...)`: Run the identify effect algorithm using defaults just provide the graph, treatment and outcome.
  * `auto_identify_effect(...)`: More configurable version of `identify_effect(...)`.
  * `id_identify_effect(...)`: Identify Effect using the ID-Algorithm.
* Refute Estimate:
  * `refute_estimate`: Function to run a set of the refuters below with the default parameters.
  * `refute_bootstrap`: Refute an estimate by running it on a random sample of the data containing measurement error in the confounders.
  * `refute_data_subset`: Refute an estimate by rerunning it on a random subset of the original data.
  * `refute_random_common_cause`: Refute an estimate by introducing a randomly generated confounder (that may have been unobserved).
  * `refute_placebo_treatment`: Refute an estimate by replacing treatment with a randomly-generated placebo variable.
  * `sensitivity_simulation`: Add an unobserved confounder for refutation (Simulation of an unobserved confounder).
  * `sensitivity_linear_partial_r2`: Add an unobserved confounder for refutation (Linear partial R2 : Sensitivity Analysis for linear models).
  * `sensitivity_non_parametric_partial_r2`: Add an unobserved confounder for refutation (Non-Parametric partial R2 based : Sensitivity Analyis for non-parametric models).
  * `sensitivity_e_value`: Computes E-value for point estimate and confidence limits. Benchmarks E-values against measured confounders using Observed Covariate E-values. Plots E-values and Observed
        Covariate E-values.
  * `refute_dummy_outcome`: Refute an estimate by introducing a randomly generated confounder (that may have been unobserved).

## Import Dependencies

In [ ]:
# Functional API imports
from dowhy.causal_identifier import (
    BackdoorAdjustment,
    EstimandType,
    identify_effect,
    identify_effect_auto,
    identify_effect_id,
)  # import effect identifier
from dowhy.causal_refuters import (
    refute_bootstrap,
    refute_data_subset,
    refute_random_common_cause,
    refute_placebo_treatment,
    sensitivity_e_value,
    sensitivity_linear_partial_r2,
    sensitivity_non_parametric_partial_r2,
    sensitivity_simulation,
    refute_dummy_outcome,
    refute_estimate,
)  # import refuters

from dowhy.causal_estimators.propensity_score_matching_estimator import PropensityScoreMatchingEstimator

from dowhy.utils.api import parse_state

from dowhy.causal_estimator import estimate_effect  # Estimate effect function

from dowhy.causal_graph import CausalGraph

# Other imports required
from dowhy.datasets import linear_dataset
from dowhy import CausalModel  # We still need this as we haven't created the functional API for effect estimation
import econml

# Config dict to set the logging level
import logging.config

DEFAULT_LOGGING = {
    "version": 1,
    "disable_existing_loggers": False,
    "loggers": {
        "": {
            "level": "WARN",
        },
    },
}


# set random seed for deterministic dataset generation 
# and avoid problems when running tests
import numpy as np
np.random.seed(1)

logging.config.dictConfig(DEFAULT_LOGGING)
# Disabling warnings output
import warnings
from sklearn.exceptions import DataConversionWarning

warnings.filterwarnings(action="ignore", category=DataConversionWarning)

## Create the Datasets

In [ ]:
# Parameters for creating the Dataset
TREATMENT_IS_BINARY = True
BETA = 10
NUM_SAMPLES = 500
NUM_CONFOUNDERS = 3
NUM_INSTRUMENTS = 2
NUM_EFFECT_MODIFIERS = 2

# Creating a Linear Dataset with the given parameters
data = linear_dataset(
    beta=BETA,
    num_common_causes=NUM_CONFOUNDERS,
    num_instruments=NUM_INSTRUMENTS,
    num_effect_modifiers=NUM_EFFECT_MODIFIERS,
    num_samples=NUM_SAMPLES,
    treatment_is_binary=True,
)

treatment_name = data["treatment_name"]
print(treatment_name)
outcome_name = data["outcome_name"]
print(outcome_name)

graph = CausalGraph(
    treatment_name=treatment_name,
    outcome_name=outcome_name,
    graph=data["gml_graph"],
    effect_modifier_names=data["effect_modifier_names"],
    common_cause_names=data["common_causes_names"],
    observed_node_names=data["df"].columns.tolist(),
)

## Identify Effect - Functional API (Preview)

In [ ]:
# Default identify_effect call example:
identified_estimand = identify_effect(graph, treatment_name, outcome_name)

# auto_identify_effect example with extra parameters:
# identified_estimand_auto = identify_effect_auto(
#     graph,
#     treatment_name,
#     outcome_name,
#     estimand_type=EstimandType.NONPARAMETRIC_ATE,
#     backdoor_adjustment=BackdoorAdjustment.BACKDOOR_EFFICIENT,
# )

# # id_identify_effect example:
# identified_estimand_id = identify_effect_id(
#     graph, treatment_name, outcome_name
# )  # Note that the return type for id_identify_effect is IDExpression and not IdentifiedEstimand

print(identified_estimand)

## Estimate Effect - Functional API (Preview)

In [ ]:
# Basic Estimate Effect function


propensity_score_estimator = PropensityScoreMatchingEstimator(
    data=data["df"],
    identified_estimand=identified_estimand,
    treatment=treatment_name,
    outcome=outcome_name,
    control_value=0,
    treatment_value=1,
    test_significance=None,
    evaluate_effect_strength=False,
    confidence_intervals=False,
    target_units="ate",
    effect_modifiers=graph.get_effect_modifiers(treatment_name, outcome_name),
)

estimate = estimate_effect(
    treatment=treatment_name,
    outcome=outcome_name,
    identified_estimand=identified_estimand,
    identifier_name="backdoor",
    method=propensity_score_estimator,
)

print(estimate)

## Refute Estimate - Functional API (Preview)

In [ ]:
# You can call the refute_estimate function for executing several refuters using default parameters
# Currently this function does not support sensitivity_* functions
refutation_results = refute_estimate(
    data["df"],
    identified_estimand,
    estimate,
    treatment_name=treatment_name,
    outcome_name=outcome_name,
    refuters=[refute_bootstrap, refute_data_subset],
)

for result in refutation_results:
    print(result)

# Or you can execute refute methods directly
# You can change the refute_bootstrap - refute_data_subset for any of the other refuters and add the missing parameters

# bootstrap_refutation = refute_bootstrap(data["df"], identified_estimand, estimate)
# print(bootstrap_refutation)

# data_subset_refutation = refute_data_subset(data["df"], identified_estimand, estimate)
# print(data_subset_refutation)

# Backwards Compatibility

This section shows replicating the same results using only the CausalModel API

In [ ]:
# Create Causal Model
causal_model = CausalModel(data=data["df"], treatment=treatment_name, outcome=outcome_name, graph=data["gml_graph"])


## Identify Effect

In [ ]:
identified_estimand_causal_model_api = (
    causal_model.identify_effect()
)  # graph, treatment and outcome comes from the causal_model object

print(identified_estimand_causal_model_api)

## Estimate Effect

In [ ]:
estimate_causal_model_api = causal_model.estimate_effect(
    identified_estimand_causal_model_api, method_name="backdoor.propensity_score_matching"
)

print(estimate_causal_model_api)

## Refute Estimate

In [ ]:
bootstrap_refutation_causal_model_api = causal_model.refute_estimate(identified_estimand_causal_model_api, estimate_causal_model_api, "bootstrap_refuter")
print(bootstrap_refutation_causal_model_api)

data_subset_refutation_causal_model_api = causal_model.refute_estimate(
    identified_estimand_causal_model_api, estimate_causal_model_api, "data_subset_refuter"
)

print(data_subset_refutation_causal_model_api)